In [3]:
import numpy as np
import pprint
import sympy as sym

pp = pprint.PrettyPrinter(indent=4)

In [4]:
AGE_COHORTS = ['0-19', '20-59', '60+']

# population fractions by region
WORLD_POP = {
    'Africa': [.507, .438, .055],
    'America': [.293, .541, .166],
    'Asia': [.312, .558, .131],
    'Europe': [.211, .532, .257]
}

# UK data from the POLYMOD survey for the above age cohorts
SAMPLES = np.array([385, 504, 122])  
CONTACT_DATA = np.array([
    [7.86, 5.22, 0.5], [2.37, 7.69, 1.06], [1.19, 5.38, 1.92]])

# Empirical disease propagation parameters in the absence of non-pharmacological intervention: 
GAMMA = 1/6.3
R0 = 2.6

A contact matrix (here, $c_{ij}$) describes the average number of daily contacts an individual in population cohort $i$ makes with people in cohort $j$. Logically contacts should satisfy a reciprocity condition, that the total number of daily contacts made by members of cohort $i$ with members of cohort $j$ should be equal to the number of contacts by cohort $j$ with cohort $i$. Which is to say, given $c_{ij}$ and a vector of population fractions $f_i$,

$$ c_{ij} f_i = c_{ji} f_j. $$

Because of variability in sampling, this doesn't hold for the above UK data, nor will it hold if we extrapolate the 
UK data to the rest of the world, but we can enforce the condition by making 
a population-weighted average the $(i,j)$ and $(j,i)$ elements of the sampled data; cf. the function symmetrize() below.

--- 

The basic reproduction rate $R_0$ is given by the largest eigenvalue of the matrix

$$ \frac{\beta}{\gamma} c_{ij} \frac{f_i}{f_j},$$

i.e. 

$$ R_0 = \rho\left(\frac{\beta}{\gamma} c_{ij} \frac{f_i}{f_j}\right),$$

where $\rho$ denotes the spectral bound (cf. [this discussion](http://sherrytowers.com/2012/12/11/sir-model-with-age-classes/#r0) and references therein). The parameter $\beta$ is the transmissibility - the probability of disease transmission given a contact between two people - and $\gamma$ is the inverse
duration of infection. We will use the above relation to estimate beta given the other parameters. As an intermediate step we will need to compute 

$$ m_{ij} =  c_{ij} \frac{f_i}{f_j},$$

which is the output of the function weight().

In [5]:
def symmetrize(pop_fracs, contact_data=CONTACT_DATA):
    """Construct a contact matrix with reciprocity from empirical data."""
    f = pop_fracs.copy()
    d = contact_data.copy()
    c = np.zeros((len(f), len(f)))

    for i in range(len(f)):
        for j in range(len(f)):
            c[i,j] = (d[i,j]*f[i] + d[j,i]*f[j])/(2*f[i])
    return c

def weight(pop_fracs, contact_matrix):
    """Weight a contact matrix by population fractions.""" 
    f = pop_fracs.copy()
    c = contact_matrix.copy()
    m = np.zeros((len(f), len(f)))

    for i in range(len(f)):
        for j in range(len(f)):
            m[i,j] = c[i,j]*f[i]/f[j]
    return m

def est_beta(pop_fracs, contact_matrix, r0, gamma):
    """Estimate disease transmissibility from known disease parameters."""
    m = weight(pop_fracs, contact_matrix)
    return r0*GAMMA / np.linalg.eigvals(m).max()

def compute_R(pop_fracs, contact_matrix, beta, gamma):
    """Compute an effective reproduction rates from known disease parameters."""
    m = weight(pop_fracs, contact_matrix)
    return np.round(beta*np.linalg.eigvals(m).max() / gamma, 2)

### Basic transmissibility

In the absence of interventions, we have:

In [42]:
c0s = {region: symmetrize(f) for region, f in WORLD_POP.items()}
m0s = {region: weight(f, c0s[region]) for region, f in WORLD_POP.items()}
beta0s = {region: est_beta(f, c0s[region], R0, GAMMA) for region, f in WORLD_POP.items()}
print('Basic contact matrices:')
pp.pprint(c0s)
print('Basic transmissibility:')
pp.pprint(beta0s)

Basic contact matrices:
{   'Africa': array([[7.86      , 3.63372781, 0.31454635],
       [4.20616438, 7.69      , 0.86778539],
       [2.89954545, 6.91072727, 1.92      ]]),
    'America': array([[7.86      , 4.79800341, 0.58709898],
       [2.59854898, 7.69      , 1.35539741],
       [1.03626506, 4.41728916, 1.92      ]]),
    'Asia': array([[7.86      , 4.72932692, 0.49982372],
       [2.64435484, 7.69      , 1.1615233 ],
       [1.19041985, 4.94755725, 1.92      ]]),
    'Europe': array([[7.86      , 5.59777251, 0.97471564],
       [2.22016917, 7.69      , 1.82949248],
       [0.80025292, 3.78712062, 1.92      ]])}
Basic transmissibility:
{   'Africa': 0.03369373971540847,
    'America': 0.03486895609630246,
    'Asia': 0.034908042242583856,
    'Europe': 0.034609209469435485}


In a vanilla SEIR model, without age structure, there is a parameter commonly denoted $\beta$ which incorporates both the transmissibility (probability of transmission given a contact) and an individual's average number of daily contacts. For the Covid-19 epidemic it is expected to be $\sim .4$. In the present model this works out to: 

In [205]:
for region, f in WORLD_POP.items():
    avg_daily_contacts = np.sum(np.dot(f, c0s[region]))
    vanilla_beta = beta0s[region]*avg_daily_contacts
    print(region)
    print('Average daily contacts: {:.1f}'.format(avg_daily_contacts))
    print('Corresponding beta of vanilla SEIR: {:.2f}'.format(vanilla_beta))

Africa
Average daily contacts: 12.2
Corresponding beta of vanilla SEIR: 0.41
America
Average daily contacts: 11.4
Corresponding beta of vanilla SEIR: 0.40
Asia
Average daily contacts: 11.6
Corresponding beta of vanilla SEIR: 0.40
Europe
Average daily contacts: 11.0
Corresponding beta of vanilla SEIR: 0.38


# Interventions

## First class
These interventions we model via an overall reduction in contacts, applied equally across age cohorts as 
a multiplicative factor $\chi < 1$ on the contact matrix. The multiple propagates straight through the eigenvalue computation.

1. Self-isolation and household quarantine. Observed reduction in R0: 37%. $\chi = .63.$
2. Self-isolation and extended contact tracing and quarantine. Observed reduction in R0: 52%. $\chi = .48.$
3. Cancellation of mass gatherings. Observed reduction in R0: 28%. $\chi = .72$.
4. Shelter in place. Observed reduction in R0 (conservative estimate): 66%. $\chi = .34$.

Resulting contact matrices:

In [29]:
npi_impacts = {
    'Quarantine': {'chi': .63},
    'Quarantine and tracing': {'chi': .48},
    'Cancel mass gatherings': {'chi': .72},
    'Shelter in place': {'chi': .34}
}

## Second class 
These inventions are age-structured.
5. School closures. We follow the per-cohort changes in contact rates from [Litvinova et al](https://www.pnas.org/content/116/27/13174).
6. Shielding the elderly. Assumed decrease in contacts between elderly and others of 50%. 

### School closures

In [43]:
impact = {'cohort_chi': [[.25, .8, 1.52],
                        [.8, .74, 1],
                        [1.52, 1, 1]]}

c_effs, R_effs = {}, {}
for region, f in WORLD_POP.items():
    c = c0s[region].copy()
    c *= impact['cohort_chi']
    c_effs.update({region: c})
    R_effs.update({region: compute_R(f, c, beta0s[region], GAMMA)})
    
print('Effective contact matrices under school closure:')
pp.pprint(c_effs)
print('Effective R0:')
pp.pprint(R_effs)
print('Fractional reductions in R0:')
reductions = {region: np.round((R0-R_effs[region])/R0, 3) for region in WORLD_POP}
pp.pprint(reductions)

npi_impacts.update({'School closure': impact})

Effective contact matrices under school closure:
{   'Africa': array([[1.965     , 2.90698225, 0.47811045],
       [3.36493151, 5.6906    , 0.86778539],
       [4.40730909, 6.91072727, 1.92      ]]),
    'America': array([[1.965     , 3.83840273, 0.89239044],
       [2.07883919, 5.6906    , 1.35539741],
       [1.57512289, 4.41728916, 1.92      ]]),
    'Asia': array([[1.965     , 3.78346154, 0.75973205],
       [2.11548387, 5.6906    , 1.1615233 ],
       [1.80943817, 4.94755725, 1.92      ]]),
    'Europe': array([[1.965     , 4.47821801, 1.48156777],
       [1.77613534, 5.6906    , 1.82949248],
       [1.21638444, 3.78712062, 1.92      ]])}
Effective R0:
{'Africa': 1.84, 'America': 1.84, 'Asia': 1.83, 'Europe': 1.86}
Fractional reductions in R0:
{'Africa': 0.292, 'America': 0.292, 'Asia': 0.296, 'Europe': 0.285}


### Shielding the elderly

In [40]:
impact = {'cohort_chi': [[1, 1, .5],
                         [1, 1, .5],
                         [.5, .5, .5]]}

c_effs, R_effs = {}, {}
for region, f in WORLD_POP.items():
    c = c0s[region].copy()
    c *= impact['cohort_chi']
    c_effs.update({region: c})
    R_effs.update({region: compute_R(f, c, beta0s[region], GAMMA)})
    
print('Effective contact matrices under shielding the elderly:')
pp.pprint(c_effs)
print('Effective R0:')
pp.pprint(R_effs)
print('Fractional reductions in R0:')
reductions = {region: np.round((R0-R_effs[region])/R0, 3) for region in WORLD_POP}
pp.pprint(reductions)

npi_impacts.update({'Shielding the elderly': impact})

Effective contact matrices under shielding the elderly:
{   'Africa': array([[7.86      , 3.63372781, 0.15727318],
       [4.20616438, 7.69      , 0.43389269],
       [1.44977273, 3.45536364, 0.96      ]]),
    'America': array([[7.86      , 4.79800341, 0.29354949],
       [2.59854898, 7.69      , 0.67769871],
       [0.51813253, 2.20864458, 0.96      ]]),
    'Asia': array([[7.86      , 4.72932692, 0.24991186],
       [2.64435484, 7.69      , 0.58076165],
       [0.59520992, 2.47377863, 0.96      ]]),
    'Europe': array([[7.86      , 5.59777251, 0.48735782],
       [2.22016917, 7.69      , 0.91474624],
       [0.40012646, 1.89356031, 0.96      ]])}
Effective R0:
{'Africa': 2.51, 'America': 2.51, 'Asia': 2.51, 'Europe': 2.5}
Fractional reductions in R0:
{'Africa': 0.035, 'America': 0.035, 'Asia': 0.035, 'Europe': 0.038}


In [38]:
pp.pprint(npi_impacts)

{   'Cancel mass gatherings': {'chi': 0.72},
    'Quarantine': {'chi': 0.63},
    'Quarantine and tracing': {'chi': 0.48},
    'School closure': {   'cohort_chi': [   [0.25, 0.8, 1.52],
                                            [0.8, 0.74, 1],
                                            [1.52, 1, 1]]},
    'Shelter in place': {'chi': 0.34},
    'Shielding the elderly': {   'cohort_chi': [   [1, 1, 0.5],
                                                   [1, 1, 0.5],
                                                   [0.5, 0.5, 0.5]]}}


## Combinations of interventions

We assume that interventions act independently, so that each added intervention imposes an additional fractional reduction to the effective contact matrix.

In [41]:
def combine(selected_npis, npi_impacts, contact_matrix):
    """Apply selected interventions to a basic contact matrix."""
    c_eff = contact_matrix.copy()
    for npi in selected_npis:
        impact = npi_impacts.get(npi, {})
        c_eff *= impact.get('chi', 1)
        c_eff *= impact.get('cohort_chi', 1)
    return c_eff

ex_combos = [['School closure', 'Shielding the elderly'],
             ['Quarantine', 'School closure'], 
             ['Quarantine and tracing', 'School closure'],
             ['Quarantine and tracing', 'School closure', 'Cancel mass gatherings'],
             ['Quarantine and tracing', 'School closure', 'Cancel mass gatherings', 'Shielding the elderly'],
             ['Shelter in place']]

for combo in ex_combos:
    print(combo)
    c_effs, R_effs = {}, {}
    for region, f in WORLD_POP.items():
        c_effs.update({region: combine(combo, npi_impacts, c0s[region])})
        R_effs.update({region: compute_R(f, c_effs[region], beta0s[region], GAMMA)})
    #print(f'c_effs: {c_effs}\n')
    print(f'R_effs: {R_effs}\n')

['School closure', 'Shielding the elderly']
R_effs: {'Africa': 1.65, 'America': 1.65, 'Asia': 1.65, 'Europe': 1.64}

['Quarantine', 'School closure']
R_effs: {'Africa': 1.16, 'America': 1.16, 'Asia': 1.15, 'Europe': 1.17}

['Quarantine and tracing', 'School closure']
R_effs: {'Africa': 0.88, 'America': 0.88, 'Asia': 0.88, 'Europe': 0.89}

['Quarantine and tracing', 'School closure', 'Cancel mass gatherings']
R_effs: {'Africa': 0.64, 'America': 0.63, 'Asia': 0.63, 'Europe': 0.64}

['Quarantine and tracing', 'School closure', 'Cancel mass gatherings', 'Shielding the elderly']
R_effs: {'Africa': 0.57, 'America': 0.57, 'Asia': 0.57, 'Europe': 0.57}

['Shelter in place']
R_effs: {'Africa': 0.88, 'America': 0.88, 'Asia': 0.88, 'Europe': 0.88}

